In [9]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import Pinecone 
import pinecone 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader 
from langchain.text_splitter  import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate 
from langchain.llms import CTransformers

In [10]:
import os
from dotenv import load_dotenv
# Load environment variables
load_dotenv()
KEY = os.getenv('PINECONE_API_KEY')
ENV=os.getenv('PINECONE_API_ENV')

In [11]:
def load_pdf(data):
     loader=DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
     documents=loader.load()
     return documents


In [12]:
extract_data=load_pdf("data/")
extract_data

[Document(metadata={'source': 'data/08. Understanding Genetics Autor Genetic Alliance.pdf', 'page': 0}, page_content='A NEW YORK – MID-ATLANTIC GUIDE FOR PATIENTS AND HEALTH PROFESSIONALS\nT h e Ne w Yo rk – Mi d - At l a n t i c C o n s o r t i u m f o r Ge n e t i c a n d Ne w b o r n S c re e n i n g Se r v i c e sUNDERSTANDING\nGENETICS'),
 Document(metadata={'source': 'data/08. Understanding Genetics Autor Genetic Alliance.pdf', 'page': 1}, page_content='Kenneth A. Pass, PhD\nNYMAC Principal Investigator, Wadsworth Center,\nNew York State Department of Health\nLouis E. Bartoshesky, MD, MPH, MALS\nNYMAC Co-Principal Investigator,\nA.I. duPont Hospital for Children\nKatharine B. Harris, MBA\nNYMAC Project Director, Wadsworth Center,\nNew York State Department of HealthBonnie L. Fredrick, MS\nNYMAC Project Coordinator, Wadsworth Center,\nNew York State Department of Health\nKate Tullis, PhD\nNYMAC Patient and Family Coordinator\nA.I. duPont Hospital for ChildrenThe New York – Mid-Atl

In [13]:
def text_split(extract_data):
   text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
   text_chunks=text_splitter.split_documents(extract_data)
   return text_chunks

In [14]:
text_chunks=text_split(extract_data)
print(text_chunks)
print('length of my chunk:',len(text_chunks))


[Document(metadata={'source': 'data/08. Understanding Genetics Autor Genetic Alliance.pdf', 'page': 0}, page_content='A NEW YORK – MID-ATLANTIC GUIDE FOR PATIENTS AND HEALTH PROFESSIONALS\nT h e Ne w Yo rk – Mi d - At l a n t i c C o n s o r t i u m f o r Ge n e t i c a n d Ne w b o r n S c re e n i n g Se r v i c e sUNDERSTANDING\nGENETICS'), Document(metadata={'source': 'data/08. Understanding Genetics Autor Genetic Alliance.pdf', 'page': 1}, page_content='Kenneth A. Pass, PhD\nNYMAC Principal Investigator, Wadsworth Center,\nNew York State Department of Health\nLouis E. Bartoshesky, MD, MPH, MALS\nNYMAC Co-Principal Investigator,\nA.I. duPont Hospital for Children\nKatharine B. Harris, MBA\nNYMAC Project Director, Wadsworth Center,\nNew York State Department of HealthBonnie L. Fredrick, MS\nNYMAC Project Coordinator, Wadsworth Center,\nNew York State Department of Health\nKate Tullis, PhD\nNYMAC Patient and Family Coordinator'), Document(metadata={'source': 'data/08. Understanding G

In [15]:
def download_hugging_face_embedings():
    embedding= HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedding

In [16]:
embedding=download_hugging_face_embedings()
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
query_result=embedding.embed_query("Generative AI")
print(query_result)
print("Length",len(query_result))


[-0.07440433651208878, -0.059408463537693024, 0.057583391666412354, 0.0006481048185378313, -0.02570318803191185, 0.022137826308608055, 0.09266568720340729, -0.015148690901696682, -0.0056587811559438705, -0.04137357696890831, -0.034168634563684464, -0.09849472343921661, 0.053848762065172195, -0.02857777662575245, -0.03763486072421074, 0.048936355859041214, 0.045807186514139175, 0.08285651355981827, -0.06154758110642433, -0.07375108450651169, -0.0027254712767899036, 0.06451703608036041, -0.017008060589432716, -0.037447571754455566, 0.046621255576610565, 0.09511933475732803, 0.02878560870885849, -0.02845599874854088, 0.08930287510156631, -0.098039411008358, 0.04297880455851555, 0.06631942093372345, -0.018005499616265297, 0.02426759898662567, -0.07755505293607712, 0.07708828896284103, -0.11995290219783783, 0.05890904739499092, 0.05904265120625496, 0.01974533125758171, -0.012411675415933132, -0.016594987362623215, 0.028167448937892914, -0.06217779219150543, 0.06355012208223343, 0.0130040226

In [28]:
Pinecone.init(api_key=KEY,
              environment=ENV)

index_name='medical-chatbot'

#creating Embedding for each of the text Chunk & storing 
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks],embedding,index_name=index_name)

SyntaxError: positional argument follows keyword argument (3880565609.py, line 1)

In [ ]:
docsearch=Pinecone.from_existing_index(index_name,embedding)

query='what is DNA'

docs=docsearch.similarity_search(query,k=3)

print("Result",docs)


In [ ]:
prompt_template="""Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=['context','question'])
chain_type_kwargs={'prompt':PROMPT}


In [ ]:
llm=CTransformers(model='OpenSource_model/llama-2-7b-chat.ggmlv3.q2_K.bin',
                  model_type='llama',
                  config={'max_new_tokens':512,
                          'temperature':0.8,
                       })



In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    retrun_source_documents=True
    chain_type_kwargs=chain_type_kwargs,
)

In [ ]:
while True:
    user_input=input(f"input prompt :")
    result=qa({'query':user_input})
    print("Response:",result['result'])